## A Step by Step Introduction to Convolutional Neural Networks
This Jupyter Notebook is intended to provide an easy-to-follow guide for implementing a convolutional neural network from scratch. The goal is to build a basic machine learning library with minimal dependencies, while also gaining an intuition for how neural networks actually work. Before we can start writing code, however, we need to understand what a neural network actually is.

To start, these are the only dependencies that we will be using:

In [274]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from PIL import Image
from keras.datasets import mnist
from keras.utils import to_categorical

### What is a neural network?
In its most basic form, a neural network is simply a function with a set of tunable parameters. This function will take in some input $x$, and will output some response $y$. The goal of the network is to accurately approximate some unknown function $f$, given a list of inputs and outputs from $f$. By adjusting the network's parameters, we can change the behavior of the network in a way that results in better approximations of $f$. This is extremely useful for approximating very complicated functions that would otherwise be unreasonably difficult to implement on a computer.

In this example, we will build a neural network $N$ that classifies images of hand-drawn digits according to which number was drawn. We will use the MNIST database, which contains a large number of 28x28 pixel black and white images of digits, each labelled with which digit is represented. In order to do this, we will train $N$ using a process called Gradient Descent. In this scenario, we can think of $f$ as the function that perfectly identifies hand-drawn digits. The data will contain a large amount of samples $(x, y)$ where $y = f(x)$. The goal is to use these samples to modify the parameters of $N$ in a way that results in similar behavior to $f$.

### Gradient Descent
Gradient Descent is an algorithm for tuning the parameters of a neural network using a cost function $C$. $C$ will generate a cost value from $f(x)$ and $N(x)$ that represents how close the predicted output from $N$ was to the true output from $f$. A lower cost means that the prediction was close to the correct result, and a cost of zero means that the prediction was exactly correct. The Gradient Descent algorithm will modify the parameters of $N$ in a way that minimizes $C$.

The parameters of $N$ can be though of as a set of numbers that affect the output of $N$. The algorithm works by passing in a list of samples in the form $(x, f(x))$. For each sample, we will compute the *gradient* of the parameters with respect to $C$. This gradient can be thought of as a list of each parameter's effect on $C$. Each value in the gradient tells us which direction to move the corresponding parameter in order to have the maximum positive effect on $C$. By moving each parameter in the opposite direction as the gradient, we will reduce the cost for the current sample.

If you have some experience with multivariable calculus, you may know that the gradient of a function is the vector of the partial derivatives of the function with respect to each of the function's parameters. In this case, the gradient is exactly the same thing. This means that in order to compute the gradient for a given sample, we need to compute the derivative of $C(N(x))$ with respect to all of the parameters of $N$.

Obviously our goal is to minimize the cost across all possible inputs, so we can't just do this for one sample. Instead, we will perform this operation across a large number of sample, making small adjustments to the parameters each time. This should result in an overall decrease in cost for each of the samples in the training data. If the training data is representative of the full set of inputs and outputs to $f$, then this should allow us to accurately predict the output of $f$ for samples that $N$ has not seen yet.

### Neural Network Structure
Before we can talk about computing the gradient, we need to discuss the actual structure of a neural network. In this example, we will construct our neural network as a list of layers. Each layer can be thought of as a function that takes its input from the previous layer and sends its output to the next layer. If we think of our layers as a list of functions $L_1, L_2, L_3$, then we can say that $N(x) = L_3(L_2(L_1))$. Due to this recursive structure, we refer to $N$ as a *recursive neural network*.

Fortunately for us, there is an efficient algorithm for computing the gradient of a recursive neural network called backpropagation. As the name implies, this algorithm involves working backwards through the layers, propagating the gradient through at each step. We can also refer to the prediction process as forward propagation, since we propagate the output from each layer to the next layer in order to get out result. The nice part about this model is that we can handle each layer independently, using only the gradient information passed backwards from the following layer.

We can model each layer using two functions:
$$
Y_i = L_i(X_i)
$$
$$
\frac{\partial C}{\partial X_i} = L_i'(\frac{\partial C}{\partial Y_i})
$$

The forward propagation function $L_i$ should be fairly straightforward. The driving idea behind $L_i'$ is that each layer can independently compute $\frac{\partial C}{\partial X_i}$ using only $\frac{\partial C}{\partial Y_i}$. This is the process of propagating gradients backwards that we just discussed. During $L_i'$, the layer will also compute the gradient for all of its internal parameters, and will adjust them accordingly. This means that we can implement each layer independently by implementing these two functions. It also means that computing the gradient and updating the parameters for the entire network is simply a matter of running through the layers backwards and calling $L_i'$ on each.

There is one more thing to go over before we can start implementing this in code. Since most of our layers will have some parameters that they will have to adjust, it is useful to understand how computing the derivatives for these parameters will work. This is where the backpropagation algorithm becomes extremely useful. In calculus, there is the concept of the chain rule, which is written as follows:

$$
\frac{\partial C}{\partial P} = \frac{\partial C}{\partial Y} \frac{\partial Y}{\partial P}
$$

Since each layer will be provided with $\frac{\partial C}{\partial P}$ during the backpropagation process, each layer can compute the derivatives for its internal parameters using only the local derivative with respect to the layer's output. This means that each layer can do all of its internal adjustments independently of the rest of the network.

Now that we have defined the structure of our neural network, we can begin to implement it. We will start with a basic abstract class for our layer. Note that we have added a rate variable to our backpropagation function in order to control the size of the parameter adjustments.

In [275]:
class Layer:
    def __init__(self):
        self.input = None

    def forward_prop(self, input):
        pass

    def backward_prop(self, d_output, rate):
        pass


Now that we have a basic structure for our network, we can start to implement different types of layers.

### Linear Layer
The first type of layer we are going to implement is the standard linear or fully connected layer. This layer has $n$ input values and $m$ output values, with each output calculated as an affine transformation of the input values. We can represent an output as follows:
$$
y_j = x_1 w_{j1} + x_2 w_{j2} + x_3 w_{j3} + \dots + x_n w_{jn} + b_j
$$

We refer to the values $w_{ij}$ as weights and the values $b_j$ as biases. Since this is an affine transformation, we can represent it as a matrix multiplication and a vector addition.

$$
\begin{bmatrix}
    y_1 \\
    y_2 \\
    \vdots \\
    y_j
\end{bmatrix}
=
\begin{bmatrix}
    w_{11} & w_{12} & \dots & w_{1i} \\
    w_{21} & w_{22} & \dots & w_{2i} \\
    \vdots & \vdots & \ddots & \vdots\\
    w_{j1} & w_{j2} & \dots & w_{ji} \\
\end{bmatrix}
\begin{bmatrix}
    x_1 \\
    x_2 \\
    \vdots \\
    x_i
\end{bmatrix}
+
\begin{bmatrix}
    b_1 \\
    b_2 \\
    \vdots \\
    b_j
\end{bmatrix}
$$

Or alternatively:

$$
Y = WX + B
$$

This gives our forward propagation calculation. For the backpropagation step, we need to make use of the derivative chain rule. There are three derivatives that we need to calculate: $\frac{\partial C}{\partial W}$, $\frac{\partial C}{\partial B}$, and $\frac{\partial C}{\partial X}$.

We will start by computing the weight gradient. We can represent the gradient of the weights as a matrix with the following form:

$$
$\frac{\partial C}{\partial W}$ =
\begin{bmatrix}
    \frac{\partial C}{\partial w_11} & \frac{\partial C}{\partial w_12} & \dots & \frac{\partial C}{\partial w_1i} \\
    \frac{\partial C}{\partial w_21} & \frac{\partial C}{\partial w_22} & \dots & \frac{\partial C}{\partial w_2i} \\
    \vdots & \vdots & \ddots & \vdots\\
    \frac{\partial C}{\partial w_j1} & \frac{\partial C}{\partial w_j2} & \dots & \frac{\partial C}{\partial w_ji} \\
\end{bmatrix}
$$

Using the chain rule from above, we can express one of these partial derivatives as follows:
$$
\frac{\partial C}{\partial w_{ji}} = \frac{\partial C}{\partial y_1} \frac{\partial y_1}{\partial w_{ji}} + \frac{\partial C}{\partial y_2} \frac{\partial y_2}{\partial w_{ji}} + \dots + \frac{\partial C}{\partial y_j} \frac{\partial y_j}{\partial w_{ji}}
$$

This looks complicated, but if we expand out the formula for $y_j$, we see that the overall expression can be simplified.

$$
y_j = x_1 w_{j1} + x_2 w_{j2} + \dots x_i w_{ji} + b_j
$$

In this expression, it is obvious that only one term contains $w_{ji}$, and this term only exists for $y_j$. Using this, we get $\frac{\partial y_j}{\partial w_{ji}} = x_i$ and $\frac{\partial y_k}{\partial w_{ji}} = 0$ for all other terms in $Y$. This means that all of our terms in the above partial derivative will cancel out to zero except one, leaving us with:

$$
\frac{\partial C}{\partial w_{ji}} = \frac{\partial C}{\partial y_j} x_i
$$

This allows us to rewrite our weights derivative matrix as a single matrix multiplication:

$$
\frac{\partial C}{\partial W} =
\begin{bmatrix}
    \frac{\partial C}{\partial y_1} \\
    \frac{\partial C}{\partial y_2} \\
    \vdots \\
    \frac{\partial C}{\partial y_j} \\
\end{bmatrix}
\begin{bmatrix}
    x_1 & x_2 & \dots & x_i
\end{bmatrix}
$$

Or, alternatively:

$$
\frac{\partial C}{\partial W} = \frac{\partial C}{\partial Y} X^T
$$

We can now move on to computing the bias gradient. This is significantly easier, and we can take a shortcut by starting with $\frac{\partial y_j}{\partial b_j}$. In this case, the computation for $y_j$ looks like the following:
$$
y_j = x_1 w_{j1} + x_2 w_{j2} + \dots + x_i + w_{ji} + b_j
$$

In this case, it is obvious that $\frac{\partial y_j}{\partial b_j} = 1$. By extending this to the other terms in $Y$, it becomes clear that for any other bias $b_k$, $\frac{\partial y_j}{\partial b_k} = 0$. By using the chain rule we end up with the following:

$$
\frac{\partial C}{\partial b_j} = \frac{\partial C}{\partial y_1} \frac{\partial y_1}{\partial b_j} + \frac{\partial C}{\partial y_2} \frac{\partial y_2}{\partial b_j} + \dots + \frac{\partial C}{\partial y_j} \frac{\partial y_1}{\partial b_j} \\

\frac{\partial C}{\partial b_j} = \frac{\partial C}{\partial y_1} 0 + \frac{\partial C}{\partial y_2} 0 + \dots + \frac{\partial C}{\partial y_j} 1 \\

\frac{\partial C}{\partial b_j} = \frac{\partial C}{\partial y_j}
$$

Or, alternatively:

$$
\frac{\partial C}{\partial B} = \frac{\partial C}{\partial Y}
$$

Finally, we can move on to computing the input gradient.

We will again start with the following:

$$
\frac{\partial C}{\partial x_i} = \frac{\partial C}{\partial y_1} \frac{\partial y_1}{\partial x_i} + \frac{\partial C}{\partial y_2} \frac{\partial y_2}{\partial x_i} + \dots + \frac{\partial C}{\partial y_i} \frac{\partial y_i}{\partial x_i}
$$

If we again look at the calculations for $Y$, we see that for $y_j$, the only term containing $x_i$ is $x_i w_{ji}$. This means that $\frac{\partial y_i}{\partial x_i} = w_{ji}$. Expanding this out to other terms in $Y$ gives us:

$$
\frac{\partial C}{\partial x_i} = \frac{\partial C}{\partial y_1} w_{1i} + \frac{\partial C}{\partial y_2} w_{2i} + \dots + \frac{\partial C}{\partial y_j} w_{ji}
$$

We can again rewrite this as a matrix multiplication:

$$
\frac{\partial C}{\partial X} = W^T \frac{\partial C}{\partial Y}
$$

Now that we have everything we need, we can implement this process in code. Let's start by collecting all of the formulas that we will need. We can express these in their matrix form as we will be implementing them with numpy.

$$
\frac{\partial C}{\partial W} = \frac{\partial C}{\partial Y} X^T \\
\frac{\partial C}{\partial B} = \frac{\partial C}{\partial Y} \\
\frac{\partial C}{\partial X} = W^T \frac{\partial C}{\partial Y} \\
$$

Notice that we have also initialized our weights and biases to random starting values. We have also implemented the weight and bias adjustments by taking the training rate into account. This will allow us to control the size of the adjustments. Also notice that we subtract the gradient from the parameters. This is because the gradient tells us which adjustments will result in the greatest increase in the cost, so we want to make the opposite adjustment.

In [276]:
class Linear(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.random.randn(output_size, 1)
        super().__init__()

    def forward_prop(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.biases

    def backward_prop(self, d_output, rate):
        d_weights = np.dot(d_output, self.input.T)
        d_biases = d_output
        d_input = np.dot(self.weights.T, d_output)

        self.weights -= d_weights * rate
        self.biases -= d_biases * rate

        return d_input

### Activation Function Layer
The next layer we will implement is an activation function. This layer is fairly simple as it just applies some non-linear function to the input. The function must be nonlinear because as shown above, the linear layer is just performing a linear transformation. If all of our layers were linear transformations then the entire network could be expressed as a giant matrix multiplication. This would make our entire network linear, which prevents it from accurately approximating non-linear functions, defeating the purpose of the neural network in the first place. The activation layer introduces a non-linear transformation, allowing the network to approximate all kinds of functions.

We can easily implement the forward propagation step for this layer, as it just involves applying a function. This means that we can express the layer as $Y = A(X)$. This layer has no parameters, so the backpropagation step is also very simple. We will only need to compute $\frac{\partial C}{\partial X}$, which can be written using the chain rule as $\frac{\partial C}{\partial X} = \frac{\partial C}{\partial Y} \frac{\partial Y}{\partial X}$. Since $Y = A(X)$, $\frac{\partial Y}{\partial X} = A'(X)$. Since we will actually be applying $A$ element-wise over $X$, we write the overall input gradient as $\frac{\partial C}{\partial X} = \frac{\partial C}{\partial Y} \circ A'(X)$, where $\circ$ represents element-wise multiplication.

The implementation for this layer is very simple, as we will take in the activation function and its derivative.

In [277]:
class Activation(Layer):
    def __init__(self, activation, d_activation):
        self.activation = activation
        self.d_activation = d_activation
        super().__init__()

    def forward_prop(self, input):
        self.input = input
        return self.activation(self.input)

    def backward_prop(self, d_output, rate):
        return np.multiply(d_output, self.d_activation(self.input))

### Sigmoid Activation Function
The first activation function we will introduce is the sigmoid function. It maps all values into the range $(0, 1)$. The specific sigmoid function we are using is called the logistic function and it is written as follows:
$$
\sigma(x)= \frac{1}{1 + e^{-x}}
$$

We aren't going to derive the derivative of the logistic function, but it can be expressed as follows:
$$
\sigma'(x) = \sigma(x) (1 - \sigma(x))
$$

The implementation in code is very simple, just remember that we are working with numpy arrays rather than single values.

In [278]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

### Mean Square Error Cost Function
The last thing we need before we can test the network is a cost function. Note that the output of the last layer in the network is the input to the cost function, so for that layer $\frac{\partial C}{\partial Y} = C'(Y)$. This means that we will need the derivative of our cost function. One simple cost function that we can use is the mean square error. As the name implies, it is the mean of the squared errors. Since the output of the network will be an array, the error is the difference between the expected result from the training data and the result generated by the network. Since the result is an array, the error will also be an array. The overall cost will be the mean of the squares of all the errors in the array.

We can easily implement this in code as follows:

In [279]:
def mse(predicted, actual):
    return np.mean(np.power(actual - predicted, 2))

def d_mse(predicted, actual):
    return 2 * (predicted - actual) / np.size(actual)

### Network Class
In order to organize our training process, we will make a class that represents our neural network. The network will have an array of layers that represent the network itself, and will have functions for computing the output of the network and training the network. We can refer to the output computation as prediction, since the network is essentially predicting the output of the function $f$ for a given input $X$.

The prediction function is fairly easy to implement, as we just pass the output from each layer into the next layer. The input to the first layer will just be the actual input to the network that we are running the prediction on.

For training the network, we will take in a loss function and its derivative, as well as our training data and some settings like the number of epochs and the learning rate. The number of epochs will just control how many times we run through the training data, and the learning rate affects how much the parameters are adjusted at each backpropagation step. The training step will be run once for each epoch, and involves keeping track of the loss for that epoch, as well as running the backpropagation step. The backpropagation step is similar to the forward propagation step, except it runs through the layers backwards and the initial input is the derivative of the cost function.

We will output the total loss accumulated over each epoch to keep track of how well the network is learning.

In [280]:
class Network:
    def __init__(self, layers):
        self.layers = layers

    def predict(self, input):
        output = input
        for layer in self.layers:
            output = layer.forward_prop(output)

        return output

    def train(self, data_x, data_y, loss, d_loss, epochs, rate):
        for e in range(epochs):
            total_loss = 0.0

            for (x, y) in zip(data_x, data_y):
                predicted = self.predict(x)

                total_loss += loss(predicted, y)

                grad = d_loss(predicted, y)
                for layer in reversed(self.layers):
                    grad = layer.backward_prop(grad, rate)

            total_loss /= len(data_x)
            print(f"Epoch {e + 1}, total loss: {total_loss}")

### Testing the Network
Before we move on to convolution, we can test the network to make sure that it works as expected. The test case that we will use is XOR, which is a simple boolean operation with two inputs and one output. This operation has the following truth table:

$$
\begin{displaymath}
\begin{array}{|c c|c|}
p & q & p \text{ XOR } q\\ % Use & to separate the columns
\hline % Put a horizontal line between the table header and the rest.
T & T & F\\
T & F & T\\
F & T & T\\
F & F & F\\
\end{array}
\end{displaymath}
$$

There are only four possible inputs, so we can easily list our training data as follows:

In [281]:
xor_training_x = np.reshape([[0, 0], [0, 1], [1, 0], [1, 1]], (4, 2, 1))
xor_training_y = np.reshape([[0], [1], [1], [0]], (4, 1, 1))

We now need to setup our layers. We will use the following layer structure:

In [282]:
xor_layers = [
    Linear(2, 3),
    Activation(sigmoid, d_sigmoid),
    Linear(3, 1),
    Activation(sigmoid, d_sigmoid),
]

Finally, we can train the network!

In [283]:
xor_network = Network(xor_layers)

xor_network.train(xor_training_x, xor_training_y, mse, d_mse, 10000, 0.1)

Epoch 1, total loss: 0.2969301073675025
Epoch 2, total loss: 0.29175529151188523
Epoch 3, total loss: 0.287124351850737
Epoch 4, total loss: 0.28303088970805473
Epoch 5, total loss: 0.2794527286053745
Epoch 6, total loss: 0.276355973338999
Epoch 7, total loss: 0.2736991082671548
Epoch 8, total loss: 0.2714366812407151
Epoch 9, total loss: 0.2695223140228045
Epoch 10, total loss: 0.2679109517586585
Epoch 11, total loss: 0.266560387208098
Epoch 12, total loss: 0.26543216632174554
Epoch 13, total loss: 0.2644920096597392
Epoch 14, total loss: 0.2637098830092901
Epoch 15, total loss: 0.26305983335517624
Epoch 16, total loss: 0.2625196828156435
Epoch 17, total loss: 0.2620706493894354
Epoch 18, total loss: 0.2616969425487391
Epoch 19, total loss: 0.2613853650440158
Epoch 20, total loss: 0.26112493978259976
Epoch 21, total loss: 0.2609065717616166
Epoch 22, total loss: 0.2607227490444178
Epoch 23, total loss: 0.2605672829475207
Epoch 24, total loss: 0.26043508533425047
Epoch 25, total loss: 

If you run the above code, you should end up with a very small total loss for the last epochs. We can test the model to make sure that it correctly predicts the results of the XOR operation.

In [284]:
print(f"0 XOR 0: predicted = {round(xor_network.predict([[0], [0]])[0][0])}, actual = 0")
print(f"0 XOR 1: predicted = {round(xor_network.predict([[0], [1]])[0][0])}, actual = 1")
print(f"1 XOR 0: predicted = {round(xor_network.predict([[1], [0]])[0][0])}, actual = 1")
print(f"1 XOR 1: predicted = {round(xor_network.predict([[1], [1]])[0][0])}, actual = 0")

0 XOR 0: predicted = 0, actual = 0
0 XOR 1: predicted = 1, actual = 1
1 XOR 0: predicted = 1, actual = 1
1 XOR 1: predicted = 0, actual = 0


If everything worked correctly the neural network should correctly replicate the behavior of the XOR operation.

### Convolution and Cross Correlation
Before we can move on to the convolutional layer, we need to understand the difference between convolutions and cross correlations. For the purpose of this guide I will assume that the reader is familiar with the basics of [image kernels](https://en.wikipedia.org/wiki/Kernel_(image_processing)). In this guide, we will refer to the kernel as a filter, and we will use the cross correlation when applying it to the image. It is important to note that a convolution can be thought of as flipping the filter upside down before sliding it along the image. A cross correlation does not flip the kernel, and is the main operation that we will be using. When it comes to handling the edge cases, we will use two kinds of operations. A *valid* cross correlation or convolution only applies the filter to pixels for which the filter does not hang over the edge of the image. This results in a filtered image that is smaller than the size of the original image. A *full* cross correlation applies the filter to all pixels and extends the image with zeros for any part of the filter that hangs off the edge.

### Convolutional Layer
The convolutional layer will apply a list of filters and biases to an image, producing one new image for each filter. The "images" are actually 3D, and have an associated depth value. This allows us to handle RGB, but in this guide we will only use grayscale images with a depth of 1. We will denote the full forward calculation as follows:

$$
Y_i = B_i + \sum_{j=1}^n X_j \circ F_{ij}, \text{   } i = 1 \dots d
$$

Where $d$ is the number of filter and bias pairs, and $n$ is the depth of the input image. $B_i$ is the bias matrix and $F_{ij}$ are the $n$ layers of the filter. $\circ$ represents the *valid* cross correlation operation. The forward propagation step will result in a new 3D image for each filter and bias pair.

Now we need to derive the backward propagation algorithm. Remember that we are given $\frac{\partial C}{\partial Y_i}$ and will need to compute $\frac{\partial C}{\partial F_{ij}}$, $\frac{\partial C}{\partial B_i}$, and $\frac{\partial C}{\partial X_j}$.

We will skip the derivations for this guide, but [this YouTube video](https://youtu.be/Lakz2MoHy6o?t=833) goes over it in more detail.

The final gradient computations are as follows:

$$
\frac{\partial C}{\partial F_{ij}} = X_j \circ \frac{\partial C}{\partial Y_i} \\
\frac{\partial C}{\partial B_i} = \frac{\partial C}{\partial Y_i} \\
\frac{\partial C}{\partial X_j} = \sum_{i = 1}^n \frac{\partial C}{\partial Y_i} \star K_{ij}
$$

Where $\star$ represents the *full convolution* operation.

We can implement this behavior as follows:

In [285]:
class Convolution(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)
        super().__init__()

    def forward_prop(self, input):
        self.input = input
        output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return output

    def backward_prop(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient

### Reshaping the Output
The output of the convolution layer is a list of images, which may not be desirable when we need to connect this layer to other layers. The solution is to make a simple layer that runs a reshape operation on the input. The backwards step for this layer is just running the same reshape in the opposite direction. The layer can be implemented as follows:

In [286]:
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
        super().__init__()

    def forward_prop(self, input):
        return np.reshape(input, self.output_shape)

    def backward_prop(self, d_output, rate):
        return np.reshape(d_output, self.input_shape)

### MNIST
Finally, we can move on to our end goal of digit recognition. The MNIST dataset is a set of labelled hand-drawn digits. Each digit is represented as a 28 x 28 array of grayscale pixel values from 0 to 255. In order to use this data set we will need to do some preprocessing. Our network will take in an image as a 28 x 28 pixel array, and will output a confidence level for each possible digit from 0 to 9. The digit chosen by the algorithm will be the digit with the highest confidence value. Since the labels in the dataset are just a number, we will need to convert it to a list of confidence values, with the correct digit having a value of 1 and all other digits having a value of 0. We will also scale the pixel values to a range from 0.0 to 1.0. This preprocessing can be implemented as follows:

In [287]:
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y) = mnist.load_data()

x = mnist_train_x
y = mnist_train_y

x = x.reshape(len(x), 1, 28, 28).astype("float32") / 255
y = to_categorical(y, num_classes=10).reshape(len(y), 10, 1)

mnist_layers = [
    Convolution((1, 28, 28), 3, 5),
    Activation(sigmoid, d_sigmoid),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Linear(5 * 26 * 26, 100),
    Activation(sigmoid, d_sigmoid),
    Linear(100, 10),
    Activation(sigmoid, d_sigmoid),
]

Now that we have setup our network, all we have to do is run it. (This may take a while)

In [288]:
mnist_network = Network(mnist_layers)

mnist_network.train(x, y, mse, d_mse, 5, 0.1)

Epoch 1, total loss: 0.0812002473938352


Assuming everything worked, you should see a steadily decreasing total loss value.

We can also test the algorithm on part of the test data.

In [290]:
test_indices = np.random.choice(len(mnist_test_y), 100, False)

print(test_indices)

test_x = mnist_test_x[test_indices]
test_y = mnist_test_y[test_indices]

test_x = test_x.reshape(len(test_x), 1, 28, 28).astype("float32") / 255
test_y = to_categorical(test_y, num_classes=10).reshape(len(test_y), 10, 1)

correct_count = 0
for (x, y) in zip(test_x, test_y):
    predicted = np.argmax(mnist_network.predict(x))
    actual = np.argmax(y)
    print(f"predicted: {predicted}, actual: {actual}")
    if actual == predicted:
        correct_count += 1

print(f"Accuracy: {correct_count / len(test_y)}")

[4850 6728  781 9968 1383 5689  871 9400 7526 9850  272 6508   27 6448
 9867 9655 4952 5448 4564  405 6276 4262  634 8040 9642 2743 2106 3595
 7177  401 3241 1199  477 1017 9583 4148 9070  231 5719 4365 6538 6088
 7668 1449 3787 7424 8273 3525 5313 9191 1715 1372 8769 4865 4767   18
 7704 8492 2210 7205 9418 1557 6609 3407 7760 9839 7874 4965 6104 4939
 7830 4818 3032 1912 9343  625 8479 7604  632 9091 4812 6945 8177 7342
 7799 5944 9797 1911 1656 2096 3853 4180 6555 4994 9305 9213 9720 4310
 5611 4415]
predicted: 8, actual: 8
predicted: 5, actual: 5
predicted: 3, actual: 8
predicted: 7, actual: 7
predicted: 3, actual: 3
predicted: 1, actual: 1
predicted: 0, actual: 0
predicted: 3, actual: 5
predicted: 0, actual: 0
predicted: 0, actual: 0
predicted: 1, actual: 1
predicted: 9, actual: 9
predicted: 4, actual: 4
predicted: 4, actual: 4
predicted: 8, actual: 2
predicted: 7, actual: 3
predicted: 6, actual: 6
predicted: 4, actual: 4
predicted: 1, actual: 1
predicted: 4, actual: 4
predicted: 

If everything worked, you should hopefully see decently accurate results. If we increase the training time the results should improve dramatically.